<a href="https://colab.research.google.com/github/GudiOP/Challenge-ONE-Telecom-X/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracción

Setup e importaciones

In [ ]:
import json, io, os, re, textwrap, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: f'{x:,.3f}')

In [ ]:
API_URL = None  # p.ej. "https://api.telecomx.com/v1/churn"
LOCAL_JSON_PATH = "/content/TelecomX_Data.json"  # súbelo a Colab si no hay API

def fetch_json_from_api(url: str, timeout=60):
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    return r.json()

def load_data():
    # 1) API si está configurada
    if API_URL:
        try:
            print("→ Extrayendo de API...")
            data = fetch_json_from_api(API_URL)
            return data
        except Exception as e:
            print(f"⚠️ Error API: {e}")

    # 2) Archivo local
    if os.path.exists(LOCAL_JSON_PATH):
        print("→ Cargando JSON local:", LOCAL_JSON_PATH)
        with open(LOCAL_JSON_PATH, "r", encoding="utf-8") as f:
            return json.load(f)

    # 3) (Opcional) Pega aquí un RAW_URL si lo tienes:
    RAW_URL = None  # p.ej. "https://raw.githubusercontent.com/.../TelecomX_Data.json"
    if RAW_URL:
        try:
            print("→ Descargando JSON de GitHub RAW...")
            txt = requests.get(RAW_URL, timeout=60).text
            return json.loads(txt)
        except Exception as e:
            print(f"⚠️ Error RAW: {e}")

    raise FileNotFoundError(
        "No se encontró fuente de datos. Configura API_URL, sube el JSON a LOCAL_JSON_PATH o define RAW_URL."
    )

raw = load_data()

# Si la raíz es una lista de registros, perfecto; si viene anidado, intenta normalizar:
if isinstance(raw, list):
    df = pd.DataFrame(raw)
elif isinstance(raw, dict):
    # intento genérico de normalización
    # busca la clave que parezca contener los registros
    candidate_keys = [k for k, v in raw.items() if isinstance(v, list) and len(v) and isinstance(v[0], dict)]
    if candidate_keys:
        df = pd.json_normalize(raw[candidate_keys[0]])
    else:
        # último recurso: normaliza todo
        df = pd.json_normalize(raw, max_level=1)
else:
    raise ValueError("Estructura JSON no reconocida.")

print("Shape:", df.shape)
df.head()

#🔧 Transformación

In [ ]:
def slugify(s):
    s = re.sub(r'[\s/]+', '_', s.strip(), flags=re.I)
    s = re.sub(r'[^a-zA-Z0-9_]+', '', s)
    return s.lower()

df.columns = [slugify(c) for c in df.columns]

# Candidatos a columna target (búsqueda flexible)
TARGET_CANDIDATES = [
    'churn','evade','evasao','evasion','abandono','customer_churn','is_churn',
    'churn_flag','churned','cancelled','cancelado','baixa'
]
target_col = None
for c in df.columns:
    if any(tc in c for tc in TARGET_CANDIDATES):
        target_col = c
        break

if target_col is None:
    print("⚠️ No se detectó automáticamente la columna objetivo (churn)."
          " Ajusta 'target_col' manualmente.")
    # EJEMPLO: target_col = 'churn'
else:
    print("Columna objetivo detectada:", target_col)

# Normalización de booleanos en todo el DF
BOOLEAN_LIKE = {"yes": True, "y": True, "si": True, "sí": True, "true": True, "1": True,
                "no": False, "n": False, "false": False, "0": False}
for c in df.columns:
    if df[c].dtype == object:
        df[c] = df[c].astype(str).str.strip()
        # intenta mapear a booleano cuando aplique
        vals = set(df[c].str.lower().unique())
        if vals.issubset(set(BOOLEAN_LIKE.keys())) or len(vals & set(BOOLEAN_LIKE.keys())) >= max(2, int(0.7*len(vals))):
            df[c] = df[c].str.lower().map(BOOLEAN_LIKE).astype('boolean')

# Conversión numérica conservadora (si es texto con números)
for c in df.columns:
    if df[c].dtype == object:
        # intenta parsear a número sin romper categorías claras
        try_num = pd.to_numeric(df[c].str.replace(',', '', regex=False), errors='coerce')
        # Si al menos 70% puede convertirse, tomamos la conversión
        if try_num.notna().mean() >= 0.7:
            df[c] = try_num

# Quitar duplicados exactos
before = len(df)
df = df.drop_duplicates()
print(f"Duplicados removidos: {before - len(df)}")

# Reporte post-limpieza
quick_profile(df)

#📊 Carga y análisis

In [ ]:
def plot_bar(series, title, top=10, rotation=30):
    counts = series.value_counts(dropna=False).head(top)
    plt.figure()
    counts.plot(kind='bar')
    plt.title(title)
    plt.xticks(rotation=rotation)
    plt.ylabel("Frecuencia")
    plt.show()

def plot_hist(series, title, bins=30):
    s = pd.to_numeric(series, errors='coerce').dropna()
    if len(s):
        plt.figure()
        plt.hist(s, bins=bins)
        plt.title(title)
        plt.xlabel(series.name)
        plt.ylabel("Frecuencia")
        plt.show()

def plot_box_by_churn(df, num_col, target):
    tmp = df[[num_col, target]].copy()
    tmp[target] = tmp[target].astype(str)
    plt.figure()
    # boxplot por clase de churn
    groups = [tmp[tmp[target]==cls][num_col].dropna().values for cls in tmp[target].unique()]
    plt.boxplot(groups, labels=tmp[target].unique())
    plt.title(f"{num_col} por {target}")
    plt.xlabel(target)
    plt.ylabel(num_col)
    plt.show()

# 6.1 Distribuciones de columnas categóricas y numéricas
cat_cols = [c for c in df.columns if df[c].dtype == 'object' or str(df[c].dtype).startswith('bool')]
num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]

for c in cat_cols[:8]:
    plot_bar(df[c], f"Distribución de {c}")

for c in num_cols[:8]:
    plot_hist(df[c], f"Histograma de {c}")

# 6.2 Churn por categoría (Top k categorías con mayor churn)
if target_col is not None:
    def churn_by_category(_df, col, target):
        g = _df.groupby(col)[target].apply(lambda s: (s.astype(str).str.lower()
                                                      .isin(['1','true','yes','y','si','sí']).mean()))
        return g.sort_values(ascending=False)

    insights_cat = {}
    for c in cat_cols:
        if c == target_col:
            continue
        try:
            g = churn_by_category(df, c, target_col)
            insights_cat[c] = g
            plt.figure()
            g.head(10).plot(kind='bar')
            plt.title(f"Churn por {c} (Top 10)")
            plt.xticks(rotation=30)
            plt.ylabel("Tasa de churn")
            plt.show()
        except Exception:
            pass

# 6.3 Churn vs. numéricos (boxplots)
if target_col is not None:
    for c in num_cols[:8]:
        try:
            plot_box_by_churn(df, c, target_col)
        except Exception:
            pass

#📄Informe final

In [ ]:
def conclusions_template(df, target):
    lines = []
    n = len(df)
    if target is not None and target in df.columns:
        y = df[target].astype(str).str.lower().isin(['1','true','yes','y','si','sí']).astype(float)
        churn = y.mean()
        lines.append(f"- Muestra analizada: {n:,} clientes.")
        lines.append(f"- Tasa general de churn: {churn:.2%}.")
    else:
        lines.append(f"- Muestra analizada: {n:,} registros. (Definir columna objetivo para tasa de churn)")

    # Top drivers si fueron calculados
    try:
        top5 = drivers.head(5).copy()
        top5['support'] = (top5['support']*100).round(1).astype(str) + '%'
        top5['churn_rate'] = (top5['churn_rate']*100).round(1).astype(str) + '%'
        lines.append("- Posibles factores asociados (EDA, no causalidad):")
        for _, r in top5.iterrows():
            lines.append(f"  • {r['feature']} = {r['value']}  → churn {r['churn_rate']} (soporte {r['support']})")
    except Exception:
        pass

    txt = "\n".join(lines + [
        "",
        "Recomendaciones iniciales (hipótesis a validar):",
        "1) Priorizar retención en segmentos con churn alto y soporte significativo.",
        "2) Revisar fricciones de experiencia (soporte, facturación, portabilidad, fallas técnicas).",
        "3) Evaluar impacto de tipo de contrato, método de pago y servicios adicionales en la evasión.",
        "4) Probar ofertas/beneficios para clientes en riesgo (cross-sell/upgrade, descuentos por permanencia).",
        "5) Entregar dataset 'model_ready' al equipo de Ciencia de Datos para modelado predictivo."
    ])
    return txt

print(conclusions_template(df, target_col))